In [ ]:
import time
import numpy as np
from gep_config_parser import *
from data_wrangling import dataframe_to_dict
from gep_main import run_model, prep_data
import pyomo.environ as pyo
import torch

DEVICE = 'cpu'

CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"

HIGHS  = "HiGHS"
GUROBI = "Gurobi"

# SAMPLE_DURATION = 12 # 12 hours
# SAMPLE_DURATION = 12 # 12 hours

## Step 0: Activate environment - ensure consistency accross computers
# print("Reading the data")
# print("Activating the environment")

## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

print("Initializing the solver")
optimizer_name = data["optimizer_config"]["solver"]

# Determine the optimizer
if optimizer_name == HIGHS:
    raise NotImplementedError(f"{optimizer_name}: Not implemented")
elif optimizer_name == GUROBI:
    
    print(f"Using {GUROBI}")
    optimizer = "gurobi_direct"
else:
    raise ValueError(f"{optimizer_name}: Not implemented")

Parsing the config file
Initializing the solver
Using Gurobi


In [12]:
for i, experiment_instance in enumerate(experiment["experiments"]):
    # Setup output dataframe
    df_res = pd.DataFrame(columns=["setup_time", "presolve_time", "barrier_time", "crossover_time", "restore_time", "objective_value"])

    T, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap = prep_data(experiment_instance)

    sample_sizes = [12, 24, 60, 120, 292, 876, 1752, 4380, 8760]
    times_per_sample_size = []
    obj_value_per_sample_size = []

    for sample_size in sample_sizes:
        T_ranges = [range(i, i + sample_size, 1) for i in range(1, len(T), sample_size)]
        objective_values = []
        times = []
        # for t in T_ranges[train_size:train_size+val_size]:
        for t in T_ranges:
            # Run one experiment for j repeats
            model, solver, time_taken = run_model(experiment_instance, t, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap)
            
            
            # print("Print values for all variables")
            # for v in model.component_data_objects(pyo.Var):
            #     print(str(v), v.value)
            objective_values.append(model.obj())
            times.append(time_taken)
        times_per_sample_size.append(np.mean(times))
        obj_value_per_sample_size.append(np.mean(objective_values))

Wrangling the input data
Populating the model
Adding model variables
Formulating the objective
Adding model constraints
Solving the optimization problem
Set parameter OutputFlag to value 1
Set parameter LogFile to value "outputs/Gurobi/output.txt"
Set parameter Crossover to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
Method  2
Crossover  0

Optimize a model with 140 rows, 113 columns and 467 nonzeros
Model fingerprint: 0x93c00021
Coefficient statistics:
  Matrix range     [7e-02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+03, 3e+04]
  RHS range        [4e+03, 3e+04]
Presolve removed 126 rows and 104 columns
Presolve time: 0.00s
Presolved: 14 rows, 9 columns, 42 nonzeros
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.900e+01
 Factor NZ  : 1.050e+02
 Factor Ops : 1.015e+03 (less tha

In [11]:
print(times_per_sample_size)

[0.011858810790597576, 0.012285008495801115, 0.013353886669629242, 0.015476974722457258, 0.04269599914550781, 0.024781298637390137, 0.05248837471008301, 0.050109028816223145, 0.09442305564880371]
